In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#out-of-focus blur

Bokeh is defined as “the effect of a soft out-of-focus background that you get when shooting a subject, using a fast lens, at the widest aperture, such as f/2.8 or wider.” Simply put, bokeh is the pleasing or aesthetic quality of out-of-focus blur in a photograph.

https://www.nikonusa.com/en/learn-and-explore/a/tips-and-techniques/bokeh-for-beginners.html

![](https://www.fullstackpython.com/img/logos/bokeh.jpg)fullstackpython

#Code by Kohei-Mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

#Thanks to Des @desalegngeb for helping me with .opts(xrotation=45) The xticks Bokeh to avoid clumsy X axis. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize,stem
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import lightgbm as lgb
import nltk
from nltk.util import ngrams
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS
import shap
shap.initjs()

In [ ]:
df = pd.read_csv("/kaggle/input/fruit-consumer-survey/Fruit_Consumer_Survey.csv")
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.rename(columns={'Age range':'age_range', 'Retail Brand':'retail_brand', 'Online channel':'online_channel'}, inplace=True)
df.head(3)

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

gender_cnt = np.round(df['Gender'].value_counts(normalize=True) * 100)
hv.Bars(gender_cnt).opts(title="Gender Count", color="green", xlabel="Gender", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=500, height=300,tools=['hover'],show_grid=True))

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

Frequence_cnt = np.round(df['Frequence'].value_counts(normalize=True) * 100)
hv.Bars(Frequence_cnt).opts(title="Frequence Count", color="green", xlabel="Frequencies", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=500, height=300,tools=['hover'],show_grid=True))\
            * hv.Text('Twice a week', 15, f"{int(Frequence_cnt.loc['Twice a week'])}%")\
            * hv.Text('Not regularly', 15, f"{int(Frequence_cnt.loc['Not regularly'])}%")\
            * hv.Text('Once a week', 15, f"{int(Frequence_cnt.loc['Once a week'])}%")\
            * hv.Text('Everyday', 15, f"{int(Frequence_cnt.loc['Everyday'])}%")\
            * hv.Text('Twice a month', 15, f"{int(Frequence_cnt.loc['Twice a month'])}%")\
            * hv.Text('Other', 15, f"{int(Frequence_cnt.loc['Other'])}%")

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

area_cnt = np.round(df['Area'].value_counts(normalize=True) * 100)
hv.Bars(area_cnt).opts(title="Area Count", color="green", xlabel="Locals", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=700, height=300,tools=['hover'],show_grid=True))

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

AREA_cnt = np.round(df['Area'].value_counts(normalize=True) * 100)
hv.Bars(AREA_cnt).opts(title="Area Percents", color="yellow", xlabel="Areas", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=500, height=300,tools=['hover'],show_grid=True))\
            * hv.Text('Northwest', 15, f"{int(AREA_cnt.loc['Northwest'])}%")\
            * hv.Text('North', 15, f"{int(AREA_cnt.loc['North'])}%")\
            * hv.Text('Northeast', 15, f"{int(AREA_cnt.loc['Northeast'])}%")\
            * hv.Text('Southwest', 15, f"{int(AREA_cnt.loc['Southwest'])}%")\
            * hv.Text('South', 15, f"{int(AREA_cnt.loc['South'])}%")\
            * hv.Text('East', 15, f"{int(AREA_cnt.loc['East'])}%")

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

age_cnt = np.round(df['age_range'].value_counts(normalize=True) * 100)
hv.Bars(age_cnt).opts(title="Age Count", color="green", xlabel="Locals", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=700, height=300,tools=['hover'],show_grid=True))

#Appling opts(xrotation=45)  the equivalent to xticks in Bokeh?

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

value_cnt = np.round(df['Value'].value_counts(normalize=True) * 100)
hv.Bars(value_cnt).opts(title="Value Count", color="red", xlabel="Values", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=700, height=300,tools=['hover'],show_grid=True)).opts(xrotation=45)

#Thanks to Des @desalegngeb  I found out how to make xticks in Bokeh

(http://holoviews.org/user_guide/Customizing_Plots.html).

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

online_cnt = np.round(df['online_channel'].value_counts(normalize=True) * 100)
hv.Bars(online_cnt[::-1]).opts(title="Online Channel Count", color="cyan", xlabel="Online Channel", ylabel="Percentage", xformatter='%d%%')\
                .opts(opts.Bars(width=600, height=600,tools=['hover'],show_grid=True,invert_axes=True))

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

super_cnt = np.round(df['Supermarket'].value_counts(normalize=True) * 100)
hv.Bars(super_cnt[::-1]).opts(title="Supermarket Count", color="purple", xlabel="Supermarket", ylabel="Percentage", xformatter='%d%%')\
                .opts(opts.Bars(width=600, height=600,tools=['hover'],show_grid=True,invert_axes=True))

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

#function to calculate ngram under several conditions

def ngram_func(ngram, trg='', trg_value=''):
    #trg_value is list-object
    if (trg == '') or (trg_value == ''):
        string_filterd =  df['retail_brand'].sum().split()
    else:
        string_filterd =  df[df[trg].isin(trg_value)]['retail_brand'].sum().split()
    dic = nltk.FreqDist(nltk.ngrams(string_filterd, ngram)).most_common(30)
    ngram_df = pd.DataFrame(dic, columns=['ngram','count'])
    ngram_df.index = [' '.join(i) for i in ngram_df.ngram]
    ngram_df.drop('ngram',axis=1, inplace=True)
    return ngram_df

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

hv.Bars(ngram_func(1)[::-1]).opts(title="Unigram Count top-30", color="red", xlabel="Unigrams", ylabel="Count")\
                .opts(opts.Bars(width=600, height=600,tools=['hover'],show_grid=True,invert_axes=True))

In [ ]:
hv.Bars(ngram_func(2)[::-1]).opts(title="Bigram Count top-30", color="yellow", xlabel="Bigrams", ylabel="Count")\
                .opts(opts.Bars(width=600, height=600,tools=['hover'],show_grid=True,invert_axes=True))

In [ ]:
hv.Bars(ngram_func(3)[::-1]).opts(title="Trigram Count top-30", color="blue", xlabel="Trigrams", ylabel="Count")\
                .opts(opts.Bars(width=600, height=600,tools=['hover'],show_grid=True,invert_axes=True))

In [ ]:
uni_ma=hv.Bars(ngram_func(1, 'Gender', ['Male'])[0:15][::-1]).opts(title="Unigram with Male", color="red", xlabel="Unigrams", ylabel="Count")
uni_fe=hv.Bars(ngram_func(1, 'Gender', ['Female'])[0:15][::-1]).opts(title="Unigram with Female", color="red", xlabel="Unigrams", ylabel="Count")

bi_ma=hv.Bars(ngram_func(2, 'Gender', ['Male'])[0:15][::-1]).opts(title="Bigram with Male", color="yellow", xlabel="Bigrams", ylabel="Count")
bi_fe=hv.Bars(ngram_func(2, 'Gender', ['Female'])[0:15][::-1]).opts(title="Bigram with Female", color="yellow", xlabel="Bigrams", ylabel="Count")

tri_ma=hv.Bars(ngram_func(3, 'Gender', ['Male'])[0:15][::-1]).opts(title="Trigram with Male", color="blue", xlabel="Trigrams", ylabel="Count")
tri_fe=hv.Bars(ngram_func(3, 'Gender', ['Female'])[0:15][::-1]).opts(title="Trigram with Female", color="blue", xlabel="Trigrams", ylabel="Count")
                

(uni_ma + uni_fe + bi_ma + bi_fe + tri_ma + tri_fe).opts(opts.Bars(width=400, height=300,tools=['hover'],show_grid=True,invert_axes=True, shared_axes=False)).opts(shared_axes=False).cols(2)

In [ ]:

wordcloud = WordCloud(width = 1500, height = 800, random_state=0, background_color='black', colormap='rainbow', \
                      min_font_size=5, max_words=300, collocations=False, min_word_length=3, stopwords = STOPWORDS).generate(" ".join(df['retail_brand'].values))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

#Feature Engineering

In [ ]:
feature_df = pd.DataFrame()
for i in [1,2,3]:
    vec_tfidf = TfidfVectorizer(max_features=10, norm='l2', stop_words='english', lowercase=True, use_idf=True, ngram_range=(i,i))
    X = vec_tfidf.fit_transform(df['retail_brand']).toarray()
    tfs = pd.DataFrame(X, columns=["TFIDF_" + n for n in vec_tfidf.get_feature_names()])
    feature_df = pd.concat([feature_df, tfs], axis=1)
feature_df = pd.concat([df, feature_df], axis=1)
feature_df.head(3)

#Label Encoding

In [ ]:
feature_df['age_range'] = LabelEncoder().fit_transform(feature_df['age_range']).astype(np.int8)
feature_df['Gender'] = LabelEncoder().fit_transform(feature_df['Gender']).astype(np.int8)
feature_df['Area'] = LabelEncoder().fit_transform(feature_df['Area']).astype(np.int8)
feature_df['Frequence'] = LabelEncoder().fit_transform(feature_df['Frequence']).astype(np.int8)
feature_df['Place'] = LabelEncoder().fit_transform(feature_df['Place']).astype(np.int8)
feature_df['Value'] = LabelEncoder().fit_transform(feature_df['Value']).astype(np.int8)
feature_df['Type'] = LabelEncoder().fit_transform(feature_df['Type']).astype(np.int8)
feature_df['retail_brand'] = LabelEncoder().fit_transform(feature_df['retail_brand']).astype(np.int8)
feature_df['Supermarket'] = LabelEncoder().fit_transform(feature_df['Supermarket']).astype(np.int8)
feature_df['online_channel'] = LabelEncoder().fit_transform(feature_df['online_channel']).astype(np.int8)
feature_df.drop(['Id','retail_brand'],axis=1,inplace=True)
feature_df.head(3)

#Train test split  had stratify =y

X_train, X_valid, Y_train, Y_valid = train_test_split(x_df, y_series, test_size=0.2, random_state=0, stratify=y_series)

I removed it to avoid error "The least populated in y has only 1 member.

In [ ]:
y_series = feature_df['Gender']
x_df = feature_df.drop(['Gender', 'online_channel'], axis=1) 
X_train, X_valid, Y_train, Y_valid = train_test_split(x_df, y_series, test_size=0.2, random_state=0 )

lgb_train = lgb.Dataset(X_train, Y_train)
lgb_valid = lgb.Dataset(X_valid, Y_valid, reference=lgb_train)

#I had to change the features to avoid Error: "label must be in [0,num_class] 

In [ ]:
params = {
    'task' : 'train',
    'boosting' : 'gbdt',
    'objective': 'multiclass',
    'num_class': 5,
    'metric': 'multi_logloss',
    'num_leaves': 200,
    'feature_fraction': 1.0,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'min_child_samples': 5
}
gbm_ac = lgb.train(params,
            lgb_train,
            num_boost_round=100,
            valid_sets=lgb_valid,
            early_stopping_rounds=100)

In [ ]:
df["online_channel"].value_counts()

#I had to change Male/Female to the online channels to avoid Error: list out of range. 

In [ ]:
ac_label = ['JD,Tmall', 'Tmall,Pinduoduo','JD', 'Meituan', 'None or Other', 'JD,Pinduoduo', 'Tmall,Suning', 'JD,Suning', 'JD,Meituan', 'Tmall', 'Pinduoduo', 'Pinduoduo,Meituan', 'Meituan,None or Other']
explainer = shap.TreeExplainer(model=gbm_ac)
shap_values_ac = explainer.shap_values(X=X_train)
shap.summary_plot(shap_values=shap_values_ac, features=X_train, feature_names=X_train.columns, plot_type="bar", max_display=30, class_names=ac_label)

In [ ]:
t = lgb.plot_tree(gbm_ac, figsize=(20, 20), precision=1, tree_index=0, show_info=['split_gain'])
plt.title('Visulalization of Tree in Online Channel')
plt.show()

#Pay attention to xrotation on the last line below. On the original code it was xrotation=0 since it was not clumsy. 

In [ ]:
f1 = lambda x : np.round(x/len(df) * 100)
gender_cnt = df.groupby(['Gender'])['age_range'].count().apply(f1)
g = hv.Bars(pd.melt(gender_cnt.reset_index(), ['Gender']), ['Gender'], 'value').opts(opts.Bars(title="Gender Count", color='green'))

f2 = lambda x : np.round(x/x.sum() * 100)
ac_gen = df.groupby(['Gender','age_range'])['age_range'].count().unstack().apply(f2, axis=1)
ac = hv.Bars(pd.melt(ac_gen.reset_index(), ['Gender']), ['Gender','age_range'], 'value').opts(opts.Bars(title="Age Range by Gender Count"))

(g + ac).opts(opts.Bars(width=400, height=300,tools=['hover'],show_grid=True,xrotation=45, ylabel="Percentage", yformatter='%d%%', shared_axes=True))

In [ ]:
uni=hv.Bars(ngram_func(1)[0:15][::-1]).opts(title="Unigram Count", color="red", xlabel="Unigrams", ylabel="Count")
bi=hv.Bars(ngram_func(2)[0:15][::-1]).opts(title="Bigram Count", color="yellow", xlabel="Bigrams", ylabel="Count")
tri=hv.Bars(ngram_func(3)[0:15][::-1]).opts(title="Trigram Count", color="blue", xlabel="Trigrams", ylabel="Count")
(uni + bi + tri).opts(opts.Bars(width=265, height=300,tools=['hover'],show_grid=True,invert_axes=True, shared_axes=False)).opts(shared_axes=False)

In [ ]:
shap.summary_plot(shap_values=shap_values_ac, features=X_train, feature_names=X_train.columns, plot_type="bar", max_display=15, class_names=ac_label)
#shap.summary_plot(shap_values=shap_values_pac, features=X_train, feature_names=X_train.columns, plot_type="bar", max_display=15, class_names=pac_label)

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Thank you Kohei-Mu, @koheimuramatsu for the code.' )